# Day3_0 별첨: 데이터클래스 (Dataclass) - 정답

이 노트북은 Day3_0 별첨 데이터클래스 실습 퀴즈의 정답 풀이를 제공합니다.

---

## Q1. 기본 데이터클래스 생성 ⭐

**문제**: `Book` 데이터클래스를 만드세요.

**요구사항**:
- 필드: `title` (제목), `author` (저자), `price` (가격), `isbn` (ISBN)
- 모든 필드는 문자열 타입
- `price`만 정수 타입

In [ ]:
# 정답
from dataclasses import dataclass

@dataclass
class Book:
    """책 정보"""
    title: str
    author: str
    price: int
    isbn: str

# 테스트
book = Book("파이썬 데이터 분석", "단테", 35000, "978-89-xxxxx")
print(book)
print(f"가격: {book.price:,}원")

### 💡 해설

- **`@dataclass` 데코레이터**: `__init__`, `__repr__`, `__eq__` 자동 생성
- **타입 힌트**: 필드 선언 시 타입 명시 (문서화 효과)
- **간결함**: 전통적 클래스 대비 70% 코드 감소

---

## Q2. 기본값과 default_factory ⭐⭐

**문제**: `ShoppingCart` 데이터클래스를 만드세요.

**요구사항**:
- 필드: `user_id` (사용자 ID, 문자열)
- 필드: `items` (상품 목록, 리스트) - 기본값은 빈 리스트
- 필드: `is_active` (활성 여부, bool) - 기본값 `True`
- 메서드: `add_item(item_name)` - 상품 추가
- 메서드: `total_items()` - 총 상품 개수 반환

In [ ]:
# 정답
from dataclasses import dataclass, field

@dataclass
class ShoppingCart:
    """장바구니"""
    user_id: str
    items: list = field(default_factory=list)  # 가변 객체는 default_factory 사용
    is_active: bool = True  # 불변 객체는 직접 기본값 설정 가능
    
    def add_item(self, item_name: str) -> None:
        """상품 추가"""
        self.items.append(item_name)
    
    def total_items(self) -> int:
        """총 상품 개수"""
        return len(self.items)

# 테스트
cart1 = ShoppingCart("user001")
cart1.add_item("노트북")
cart1.add_item("마우스")

cart2 = ShoppingCart("user002")
cart2.add_item("키보드")

print(f"cart1: {cart1.items}, 총 {cart1.total_items()}개")
print(f"cart2: {cart2.items}, 총 {cart2.total_items()}개")
print(f"독립적? {cart1.items is not cart2.items}")

### 💡 해설

- **`field(default_factory=list)`**: 각 인스턴스마다 새로운 리스트 생성
- **가변 vs 불변 기본값**:
  - 불변 (int, str, bool): `is_active: bool = True` (직접 설정 가능)
  - 가변 (list, dict, set): `field(default_factory=list)` (팩토리 함수 필요)
- **주의**: `items: list = []`는 에러 발생! (모든 인스턴스가 같은 리스트 공유)

---

## Q3. __post_init__으로 계산된 필드 ⭐⭐⭐

**문제**: `Circle` 데이터클래스를 만드세요.

**요구사항**:
- 필드: `radius` (반지름, float)
- 계산 필드: `area` (넓이) - `__post_init__`에서 계산 (π * r²)
- 계산 필드: `circumference` (둘레) - `__post_init__`에서 계산 (2 * π * r)
- `area`와 `circumference`는 `init=False`로 설정
- π는 3.14159 사용

In [ ]:
# 정답
from dataclasses import dataclass, field

@dataclass
class Circle:
    """원"""
    radius: float
    
    # init=False: __init__에서 받지 않음 (자동 계산)
    area: float = field(init=False)
    circumference: float = field(init=False)
    
    def __post_init__(self):
        """인스턴스 생성 후 자동 호출"""
        pi = 3.14159
        self.area = pi * self.radius ** 2
        self.circumference = 2 * pi * self.radius

# 테스트
circle = Circle(5.0)
print(circle)
print(f"넓이: {circle.area:.2f}")
print(f"둘레: {circle.circumference:.2f}")

### 💡 해설

- **`field(init=False)`**: `__init__` 메서드에 해당 필드를 매개변수로 받지 않음
- **`__post_init__`**: 인스턴스 생성 후 자동으로 호출되는 메서드
  - 다른 필드 값으로부터 계산된 값을 설정할 때 사용
  - 유효성 검증에도 활용 가능
- **패턴**: `radius`만 입력받아 `area`, `circumference` 자동 계산

**대안 방법**: `@property` 사용
```python
@dataclass
class Circle:
    radius: float
    
    @property
    def area(self):
        return 3.14159 * self.radius ** 2
```

---

## Q4. order=True로 정렬 ⭐⭐⭐

**문제**: `Product` 데이터클래스를 가격순으로 정렬 가능하게 만드세요.

**요구사항**:
- 필드: `name` (제품명, str), `price` (가격, int), `stock` (재고, int)
- `order=True` 설정
- `sort_index` 필드 추가 (가격순 정렬용, init=False, repr=False)
- `__post_init__`에서 `sort_index = self.price` 설정
- 리스트를 정렬하면 가격 낮은 순으로 정렬되어야 함

In [ ]:
# 정답
from dataclasses import dataclass, field

@dataclass(order=True)
class Product:
    """제품 (가격순 정렬)"""
    # 정렬 기준 필드 (첫 번째 필드가 정렬 기준)
    sort_index: int = field(init=False, repr=False)
    
    name: str
    price: int
    stock: int
    
    def __post_init__(self):
        # 가격을 정렬 기준으로 설정
        self.sort_index = self.price

# 테스트
products = [
    Product("노트북", 1500000, 10),
    Product("마우스", 25000, 50),
    Product("키보드", 89000, 30),
    Product("모니터", 350000, 15),
]

print("=== 가격순 정렬 ===")
for p in sorted(products):
    print(f"{p.name}: {p.price:,}원")

### 💡 해설

- **`@dataclass(order=True)`**: `<`, `<=`, `>`, `>=` 비교 연산자 자동 생성
- **정렬 기준**: 필드 순서대로 비교 (첫 번째 필드 우선)
- **`sort_index` 패턴**:
  - `init=False`: `__init__`에서 받지 않음
  - `repr=False`: 출력에서 숨김
  - 첫 번째 필드로 배치하여 정렬 기준으로 사용
- **역순 정렬**: `sort_index = -self.price` (음수 사용)

**대안 방법**: `sorted()` 함수의 `key` 매개변수
```python
sorted(products, key=lambda p: p.price)
```

---

## Q5. frozen=True 불변 객체 ⭐⭐⭐⭐

**문제**: `Config` 데이터클래스를 불변으로 만들고 딕셔너리 키로 사용하세요.

**요구사항**:
- 필드: `env` (환경, str), `debug` (디버그 모드, bool)
- `frozen=True` 설정
- 생성 후 값 변경 시도하면 에러 발생
- 딕셔너리 키로 사용 가능 (해시 가능)

In [ ]:
# 정답
from dataclasses import dataclass

@dataclass(frozen=True)
class Config:
    """설정 (불변)"""
    env: str
    debug: bool

# 테스트
dev_config = Config("development", True)
prod_config = Config("production", False)

# 딕셔너리 키로 사용
settings = {
    dev_config: {"db_host": "localhost", "cache": False},
    prod_config: {"db_host": "prod.db.com", "cache": True},
}

print(f"개발 환경 설정: {settings[dev_config]}")
print(f"프로덕션 환경 설정: {settings[prod_config]}")

# 수정 시도 (에러 발생 확인)
try:
    dev_config.debug = False
except Exception as e:
    print(f"\n예상된 에러: {type(e).__name__}")
    print("frozen=True로 설정되어 값을 변경할 수 없습니다!")

### 💡 해설

- **`@dataclass(frozen=True)`**: 불변 객체 생성
  - 생성 후 필드 값 변경 불가 (`FrozenInstanceError` 발생)
  - 튜플처럼 동작 (immutable)
  
- **해시 가능 (Hashable)**:
  - `frozen=True`이면 `__hash__()` 자동 생성
  - 딕셔너리 키, 집합 원소로 사용 가능
  
- **frozen=True의 장점**:
  1. **스레드 안전**: 동시성 환경에서 안전하게 공유
  2. **의도 명확**: "이 객체는 변경되지 않는다"는 의도 표현
  3. **버그 방지**: 의도하지 않은 수정 차단
  
- **실무 활용**:
  - 설정값 (Config)
  - 좌표 (Point, Coordinate)
  - 날짜/시간 (Date, Time)
  - 상수 집합

---

## Q6. 종합 실무 예제 ⭐⭐⭐⭐⭐

**문제**: 로그 분석기 데이터클래스를 만드세요.

**요구사항**:

**LogEntry 데이터클래스**:
- 필드: `timestamp` (str), `level` (str), `message` (str)
- `level`은 "INFO", "WARNING", "ERROR" 중 하나

**LogAnalyzer 데이터클래스**:
- 필드: `source` (소스명, str)
- 필드: `logs` (로그 목록, List[LogEntry]) - default_factory=list
- 필드: `stats` (통계, dict) - init=False
- 메서드: `add_log(timestamp, level, message)` - 로그 추가
- 메서드: `analyze()` - 로그 레벨별 개수를 stats에 저장
- 메서드: `get_errors()` - ERROR 로그만 반환
- `__post_init__`에서 `self.stats = {}` 초기화

In [ ]:
# 정답
from dataclasses import dataclass, field
from typing import List

@dataclass
class LogEntry:
    """로그 항목"""
    timestamp: str
    level: str  # "INFO", "WARNING", "ERROR"
    message: str

@dataclass
class LogAnalyzer:
    """로그 분석기"""
    source: str
    logs: List[LogEntry] = field(default_factory=list)
    stats: dict = field(init=False, default_factory=dict)
    
    def add_log(self, timestamp: str, level: str, message: str) -> None:
        """로그 추가"""
        log = LogEntry(timestamp, level, message)
        self.logs.append(log)
    
    def analyze(self) -> None:
        """로그 레벨별 개수 분석"""
        self.stats = {"INFO": 0, "WARNING": 0, "ERROR": 0}
        
        for log in self.logs:
            if log.level in self.stats:
                self.stats[log.level] += 1
    
    def get_errors(self) -> List[LogEntry]:
        """ERROR 로그만 반환"""
        return [log for log in self.logs if log.level == "ERROR"]

# 테스트
analyzer = LogAnalyzer("app.log")

analyzer.add_log("2024-01-15 10:00:00", "INFO", "서버 시작")
analyzer.add_log("2024-01-15 10:05:30", "INFO", "사용자 로그인")
analyzer.add_log("2024-01-15 10:10:15", "WARNING", "메모리 사용량 80% 초과")
analyzer.add_log("2024-01-15 10:15:00", "ERROR", "데이터베이스 연결 실패")
analyzer.add_log("2024-01-15 10:20:00", "ERROR", "API 요청 타임아웃")
analyzer.add_log("2024-01-15 10:25:00", "INFO", "정상 복구")

# 분석
analyzer.analyze()
print(f"총 로그 수: {len(analyzer.logs)}개")
print(f"통계: {analyzer.stats}")

# 에러 로그만 출력
print("\n=== 에러 로그 ===")
for log in analyzer.get_errors():
    print(f"[{log.timestamp}] {log.message}")

### 💡 해설

**LogEntry 클래스**:
- 단순 데이터 구조체 (DTO 패턴)
- 로그 한 줄의 정보를 구조화

**LogAnalyzer 클래스**:
- **`logs: List[LogEntry]`**: 타입 힌트로 명확한 관계 표현
- **`field(default_factory=list)`**: 각 인스턴스마다 독립적인 리스트
- **`field(init=False, default_factory=dict)`**: `stats`는 `__init__`에서 받지 않고, 빈 딕셔너리로 초기화
- **`analyze()` 메서드**: 로그를 순회하며 레벨별 카운팅
- **`get_errors()` 메서드**: 리스트 컴프리헨션으로 필터링

**핵심 패턴**:
1. **조합 (Composition)**: LogAnalyzer가 LogEntry 리스트를 포함
2. **타입 힌트**: `List[LogEntry]`로 자료구조 명확화
3. **분리된 책임**: LogEntry는 데이터만, LogAnalyzer는 분석 로직

**실무 확장**:
```python
# 1. 시간 범위 필터
def get_logs_between(self, start: str, end: str):
    return [log for log in self.logs 
            if start <= log.timestamp <= end]

# 2. 로그 파일 저장
def save_to_file(self, filepath: str):
    with open(filepath, 'w') as f:
        for log in self.logs:
            f.write(f"{log.timestamp} [{log.level}] {log.message}\n")

# 3. JSON 직렬화
from dataclasses import asdict
def to_json(self):
    return json.dumps([asdict(log) for log in self.logs], indent=2)
```

---

## 🎯 주요 학습 포인트

### 1. 가변 기본값 처리
```python
# ❌ 잘못된 방법
items: list = []

# ✅ 올바른 방법
items: list = field(default_factory=list)
```

### 2. 계산된 필드
```python
# init=False + __post_init__
calculated: float = field(init=False)

def __post_init__(self):
    self.calculated = self.x * self.y
```

### 3. 정렬 가능한 데이터클래스
```python
@dataclass(order=True)
class Item:
    sort_index: int = field(init=False, repr=False)
    # ... 다른 필드들
    
    def __post_init__(self):
        self.sort_index = self.priority
```

### 4. 불변 객체
```python
@dataclass(frozen=True)
class ImmutableConfig:
    # 딕셔너리 키로 사용 가능
    # 스레드 안전
    # 의도하지 않은 변경 방지
```

### 5. 실무 활용 패턴
- **DTO (Data Transfer Object)**: API 요청/응답
- **Configuration**: 설정 관리 (frozen=True)
- **Analysis Result**: 데이터 분석 결과
- **Domain Model**: 비즈니스 객체
- **Composite Pattern**: 여러 데이터클래스 조합

---

## 📚 더 알아보기

### asdict()와 astuple()
```python
from dataclasses import asdict, astuple

@dataclass
class User:
    name: str
    age: int

user = User("단테", 30)

# 딕셔너리로 변환
print(asdict(user))  # {'name': '단테', 'age': 30}

# 튜플로 변환
print(astuple(user))  # ('단테', 30)
```

### replace()
```python
from dataclasses import replace

# frozen=True인 객체의 "복사본" 생성 (일부 필드만 변경)
@dataclass(frozen=True)
class Point:
    x: int
    y: int

p1 = Point(1, 2)
p2 = replace(p1, x=10)  # Point(10, 2)
```

**수고하셨습니다!** 🎉